In [76]:
import json
import requests
from os import path, makedirs
from datetime import date
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import pandas as pd
import re


keyword = '스토킹'
base_dir = path.dirname('__file__')
data_dir = path.join(base_dir, 'data')
today = date(1999, 12, 31)




for i in range(24):
    diff = relativedelta(months=i)
    this_month_start = date(today.year, today.month, 1) - diff
    _, end_date = monthrange(this_month_start.year, this_month_start.month)
    this_month_end = date(this_month_start.year, this_month_start.month, end_date)
    print(this_month_start)
    #dir_name = f'{this_month_start.year}-{this_month_start.month}'
    #dir_path = path.join(data_dir, dir_name)
    
    
    if not path.exists(data_dir):
        makedirs(data_dir)
        
    page = 0    
    final_result = pd.DataFrame(columns=['date', 'title', 'source', 'contents', 'link', 'writer', 'incident_category', 'category'])
    
    
    while True:
        page += 1
        response = requests.post('https://www.bigkinds.or.kr/api/news/search.do', headers={
            'Referer': 'https://www.bigkinds.or.kr/v2/news/search.do',
            'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
        }, json={
            'indexName': 'news',
            'searchKey': keyword,
            'startDate': str(this_month_start),
            'endDate': str(this_month_end),
            'startNo': page,
            'resultNumber': 100,
        })
        
        data = response.json()['resultList']
        
        title_text = []
        link_text = []
        source_text = []
        date_text = []
        contents_text = []
        writer_text = []
        incident_category_text = []
        category = []       
        
        n = len(data)
        if n < 1:
            break
            
        for news in data:
            if (news['CATEGORY_NAMES'].find('경제') != -1) or (news['CATEGORY_NAMES'].find('방송_연예') != -1) or (news['CATEGORY_NAMES'].find('공연') != -1):
                continue
            elif not news['NEWS_ID']: continue    
            else:
                title_text.append(news['TITLE'])
                link_text.append(news['PROVIDER_LINK_PAGE'])
                source_text.append(news['PROVIDER'])
                date_text.append(news['DATE'])
                writer_text.append(news['BYLINE'])
                incident_category_text.append(re.sub('<.+?>', '',news['CATEGORY_INCIDENT_NAMES'], 0).strip())
                category.append(re.sub('<.+?>', '',news['CATEGORY_NAMES'], 0).strip())
                news_id = news['NEWS_ID']


                response2 = requests.get('https://www.bigkinds.or.kr/news/detailView.do?docId='+news_id+'&returnCnt=1&sectionDiv=1000', headers={
                    'Referer': 'https://www.bigkinds.or.kr/v2/news/search.do', 'User-Agent': 'Request'})

                contents_text.append(re.sub('<.+?>', '', response2.json()['detail']['CONTENT'], 0).strip())

            
        
        result= {"date" : date_text , "title":title_text , "source" : source_text ,"contents": contents_text ,"link":link_text, 
                 "writer": writer_text, "incident_category": incident_category_text, "category": category}
        
        
        
        df = pd.DataFrame(result)
        
        
        final_result = pd.concat([final_result, df], ignore_index=True)
        
        # 한 달치를 파일 하나로 저장
        
        
    #print(results)
    #print(final_result)
    
    final_result.to_csv(path.join(data_dir ,str(this_month_start).replace('-', '')[:-2]+'.csv'), index=False)
    

print('end')
    #file_path = path.join(dir_path, 'data.json')
    #with open(file_path, 'w+') as f:
    #    json.dump(results, f)

1999-12-01
1999-11-01
1999-10-01
1999-09-01
1999-08-01
1999-07-01
1999-06-01
1999-05-01
1999-04-01
1999-03-01
1999-02-01
1999-01-01
1998-12-01
1998-11-01
1998-10-01
1998-09-01
1998-08-01
1998-07-01
1998-06-01
1998-05-01
1998-04-01
1998-03-01
1998-02-01
1998-01-01
end
